# EV State-Space Model

This  notebook is used to reproduce the State Space Model of aggregateed EV for frequency regulation.

Running on Jinning's local machine, in the env "ev".

Working notes:

02/06/2022: the EV data generation is completed. It looks weired, we may need update it later on.


In [1]:
import andes
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger(__name__)

print(f"ANDES Version: {andes.__version__}")

ANDES Version: 1.6.5.post60+ga84a2f4e


## EV data

### Define parameters

The EV parameters are defined as two types. Type I follows uniform distribution, which is stored in a Dict `ev_param`. Type II follows normal distribution, which is stored in a pd.DataFrame `ev_pdf`.

The data are cited from:

M. Wang et al., "State Space Model of Aggregated Electric Vehicles for Frequency Regulation," in IEEE Transactions on Smart Grid, vol. 11, no. 2, pp. 981-994, March 2020, doi: 10.1109/TSG.2019.2929052.

In [2]:
import scipy.stats as stats
import random
import itertools


def find_x(x, soc_intv):
    for idx in soc_intv.keys():
        if x >= soc_intv[idx][0] and x <= soc_intv[idx][1]:
            return idx


def update_xl(inl):
    inl = list(inl).copy()
    dc = inl[1].copy()
    if inl[2]*inl[3] == 1:
        dc[len(dc)-1].append(inl[0])
    else:
        dc[len(dc)] = [inl[0]]
    return True


class ev_ssm():
    """
    EV State Space Model.

    EV parameters:
    Pc, Pd, nc, nd, En follow uniform distribution.
    soci, socd, ts, tf follows normal distribution.

    Attributes
    ----------
    ev: pandas.DataFrame
        EV dataset
        u: online status
        Pc/Pd: charging/discharging power (kW)
        nc/nd: charging/discharging efficiency
        En: capacity (kWh)
        soci/socd: initial/demand SOC (unit: %)
        ts/tf: start/finish time (unit: 24H)
        socx: SOC interval
    tnow: float
        current time (unit: 24H)
    N: int
        Total number of EVs
    Ns: int
        Total number of SoC intervals
    ne: int
        Online number of EVs

    Notes
    -----
    ev_ufparam:
        Pl: rated charging/discharging power (kW) lower bound
        Pu: rated charging/discharging power (kW) upper bound
        nl: charging/discharging efficiency lower bound
        nu: charging/discharging efficiency upper bound
        Enl: Battery capacity (kWh) lower bound
        Enu: Battery capacity (kWh) upper bound
        socl: Minimum SoC value
        socu: Maximum SoC value
    ev_nfparam:
        soci: initial SoC
        socd: demanded SoC
        ts1: start charging time, [24H]
        ts2: start charging time, [24H]
        tf1: finish charging time, [24H]
        tf2: finish charging time, [24H]
    """

    def find_socx(self):
        self.ev['socx'] = self.ev['soc'].apply(lambda x: find_x(x, self.soc_intv))

    def build(self):
        """
        Build the ev DataFrame.

        Returns
        -------
        ev: pandas.DataFrame
            EV dataset
        """
        self.socl = self.ev_ufparam['socl']
        self.socu = self.ev_ufparam['socu']

        #  --- 1a. uniform distribution parameters range ---
        unit = self.ev_ufparam['socu']/self.ev_ufparam['Ns']
        self.soc_intv = {}
        decimal = 4
        for i in range(self.ev_ufparam['Ns']):
            intv_single = [np.around(i*unit, decimal), np.around((i+1)*unit, decimal)]
            self.soc_intv[i] = intv_single
        self.Ns = self.ev_ufparam['Ns']
        ev_pdf = pd.DataFrame(data=self.ev_pdf_data, index=self.ev_pdf_name).transpose()
        self.ev_nfparam = ev_pdf.to_dict()

        # --- 1c. generate EV dataset ---
        self.ev = pd.DataFrame()
        np.random.seed(self.seed)

        #  data from uniform distribution
        cols = ['Pc', 'Pd', 'nc', 'nd', 'En']
        cols_bound = {'Pc':   ['Pl', 'Pu'],
                      'Pd':   ['Pl', 'Pu'],
                      'nc':   ['nl', 'nu'],
                      'nd':   ['nl', 'nu'],
                      'En':    ['Enl', 'Enu']}
        for col in cols:
            idxl = cols_bound[col][0]
            idxh = cols_bound[col][1]
            self.ev[col] = np.random.uniform(
                low=self.ev_ufparam[idxl],
                high=self.ev_ufparam[idxh],
                size=self.N)

        #  data from normal distribution
        # soci, socd
        for col in self.ev_pdf_name:
            self.ev[col] = stats.truncnorm(
                (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(self.N)
        self.ev['soc'] = self.ev['soci']
        # ts1, ts2, tf1, tf2
        et = self.ev.copy()
        r1 = 0.5  # ratio of t1
        tp1 = self.ev[['ts1', 'tf1']].sample(n=int(et.shape[0]*r1), random_state=2021)
        tp2 = self.ev[['ts2', 'tf2']].sample(n=int(et.shape[0]*(1-r1)), random_state=2021)
        tp = pd.concat([tp1, tp2], axis=0).reset_index(drop=True).fillna(0)
        tp['ts'] = tp['ts1'] + tp['ts2']
        tp['tf'] = tp['tf1'] + tp['tf2']
        check = tp.ts > tp.tf
        row_idx = tp[check].index
        mid = tp.tf.iloc[row_idx].values
        tp.tf.iloc[row_idx] = tp.ts.iloc[row_idx]
        tp.ts.iloc[row_idx] = mid
        check = tp.ts > tp.tf
        self.ev['ts'] = tp['ts']
        self.ev['tf'] = tp['tf']
        self.ev['u'] = 1
        self.ev = self.ev[['u', 'Pc', 'Pd', 'nc', 'nd', 'En', 'soc', 'soci', 'socd', 'ts', 'tf']]

        # Initiallize control signal
        self.ev['ctrl'] = random.choices([-1, 0, 1], k=self.N, weights=[0.1, 0.1, 0.8])

        # Initialize delta power
        self.ev['dP'] = 0

        self.states = list(itertools.product([1, 0, -1], self.soc_intv.keys()))
        # --- update soc interval and online status ---
        self.g_u()
        self.g_ctrl()
        self.find_socx()
        self.g_xs()
        self.g_x()
        # initialize x series
        self.ev['xl'] = self.ev['x'].apply(lambda x: {0: [x]})
        self.ne = self.ev.u.sum()

        self.ssm_basic()

    def report(self):
        """
        Report EVA.
        """
        # --- EV summary info ---
        self.En = self.ev.En.sum()/1e3
        self.wEn = np.sum(self.ev.u * self.ev.En)/1e3
        # --- report info ---
        msg1 = f"{self.name}:\n"
        msg_time = f'tnow={np.round(self.tnow, 4)} [24H]\n'
        msg_ev = f"{self.N} EVs, {self.ne} online, Total En={self.En.round(2)} MWh, SoC intervals: {len(self.soc_intv)}\n"
        msg_soc = f"Online En={self.wEn.round(2)} MWh, mean SoC={100*self.ev.soc.mean().round(2)}%"
        logger.warning(msg1 + msg_time + msg_ev + msg_soc)

    def __init__(self, tnow=0, N=20000, seed=2021, name="EVA"):
        """
        Parameters
        ----------
        N: int
            Number of EVs
        seed: int
            Random seed
        """
        # --- 1. init ---
        self.name = name
        self.tnow = tnow  # time now
        self.ts = [self.tnow]
        self.N = N
        self.seed = seed
        # --- 1a. uniform distribution parameters range ---
        self.ev_ufparam = dict(Ns=20,
                               Pl=5.0, Pu=7.0,
                               nl=0.88, nu=0.95,
                               Enl=20.0, Enu=30.0,
                               socl=0, socu=1)
        #  --- 1b. normal distribution parameters range ---
        self.ev_pdf_name = ['soci', 'socd', 'ts1', 'ts2', 'tf1', 'tf2']
        self.ev_pdf_data = {'mean':     [0.3,    0.8,    -6.5,   17.5,   8.9,    32.9],
                            'var':      [0.05, 0.03, 3.4, 3.4, 3.4, 3.4],
                            'lb':       [0.2, 0.7, 0.0, 5.5, 0.0, 20.9],
                            'ub':       [0.4, 0.9, 5.5, 24.0, 20.9, 24.0],
                            'info':  ['initial SoC', 'demanded SoC',
                                      'start charging time 1', 'start charging time 2',
                                      'finish charging time 1', 'finish charging time 2']}

        self.build()
        self.report()

        # --- SSM ---
        # --- input: AGC signal ---

        # --- output: estimated FRC ---
        self.prumax = 0
        self.prdmax = 0

    def act(self, t=4, tnow=10):
        """
        Response of the EV_SSM to the control signal.

        Parameters
        ----------
        t: int
            Action time (second).
        tnow: int
            current time (hour with decimals).
        """
        self.ts.append(tnow)
        self.tnow = tnow
        self.g_u(tnow=tnow)  # update online status
        self.g_ctrl()  # update control signal

        # --- update soc interval and online status ---
        # charging/discharging power, kW
        self.ev['dP'] = self.ev[['Pc', 'Pd', 'nc', 'nd', 'ctrl', 'u']].apply(
            lambda x: x[0]*x[2]*x[5] if x[4] >= 0 else -1*x[1]*x[3]*x[5], axis=1)
        self.ev['dP'] = self.ev['dP'] * self.ev['u']  # consider online status
        self.ev['soc'] = self.ev.soc + t/3600 * self.ev['dP'] / self.ev['En']  # update SoC
        self.ev['soc'] = self.ev['soc'].apply(lambda x: x if x < self.socu else self.socu)
        self.ev['soc'] = self.ev['soc'].apply(lambda x: x if x > self.socl else self.socu)

        # update x
        self.find_socx()
        self.g_xs()
        self.g_x()
        self.g_xl()

        # update tnow
        self.tnow = self.tnow + t/3600

    def g_u(self, tnow=10):
        """
        Update online status of EV at given time.

        Parameters
        ----------
        tnow: int
            current time (hour with decimals).
        """
        self.t0 = self.tnow
        self.ev['u0'] = (self.ev.ts <= self.t0) & (self.ev.tf >= self.t0)
        self.ev['u0'] = self.ev['u0'].astype(int)
        self.tnow = tnow
        self.ev['u'] = (self.ev.ts <= self.tnow) & (self.ev.tf >= self.tnow)
        self.ev['u'] = self.ev['u'].astype(int)
        self.ne = self.ev.u.sum()

    def g_ctrl(self):
        """
        Generate the charging signal.
        """
        # --- default signal is charging for all ---
        # TODO: replace with SSM later on
        # self.ev['ctrl'] = np.ones(self.N)
        # random charging/ discharging
        self.ev['ctrl'] = random.choices([-1, 1], k=self.N, weights=[0.2, 0.8])
        # revise control
        self.ev['ctrl'] = self.ev[['soc', 'ctrl']].apply(
            lambda x: 0 if x[0] >= 0.9 else x[1], axis=1)  # fully charged EVs set as idle state
        self.ev['ctrl'] = self.ev[['soc', 'ctrl']].apply(
            lambda x: 1 if x[0] <= 0.1 else x[1], axis=1)  # low charged EVs set as charging state
        self.ev['ctrl'] = self.ev[['ctrl', 'u']].apply(
            lambda x: x[0]*x[1], axis=1)  # offline EVs set as idle state
        self.ev['ctrl'] = self.ev['ctrl'].astype(int)

    def reset(self, tnow=10):
        """
        Reset the EV_SSM to before charging state.

        Parameters
        ----------
        tnow: float
            current time (hour with decimals).
        """
        self.ev.soc = self.ev.soci
        self.tnow = tnow
        self.g_u()
        self.find_socx()
        self.g_x()
        self.ev['xl'] = self.ev['x'].apply(lambda x: {0: [x]})
        self.ts = [tnow]
        logger.warning(f"{self.name}: Successfully reset to time={self.tnow}")

    def g_xs(self):
        """
        Update SSM X.
        """
        self.x = {}
        for s in self.states:
            n = self.ev[(self.ev.ctrl == s[0]) & (self.ev.socx == s[1])].shape[0]
            self.x[str(s[1])+'S'+str(s[0])] = n / self.N

    def g_x(self):
        """
        Update EV x.
        """
        self.ev['x'] = self.ev['socx'].astype(str) + 'S' + self.ev['ctrl'].astype(str)
        # change str to number
        self.ev['x'] = self.ev.x.replace(self.x.keys(), range(60))

    def g_xl(self):
        """
        Update EV x series.
        """
        self.ev[['x', 'xl', 'u', 'u0']].apply(lambda x: update_xl(x), axis=1)

    def ssm_basic(self):
        """
        Build SSM B, C, D matrix.
        """
        B1 = -1 * np.eye(self.Ns)
        B2 = np.eye(self.Ns)
        B3 = np.zeros((self.Ns, self.Ns))
        self.B = np.vstack((B1, B2, B3))

        C1 = np.zeros((self.Ns, self.Ns))
        C2 = -1 * np.eye(self.Ns)
        C3 = np.eye(self.Ns)
        self.C = np.vstack((C1, C2, C3))

        # P average
        kde = stats.gaussian_kde(self.ev.Pc)
        Pave = 0  # TODO: consider Pave as an attribute
        step = 0.01
        for Pl in np.arange(self.ev.Pc.min(), self.ev.Pc.max(),step):
            Pave += (kde.integrate_box(Pl, Pl+step)) * (Pl + 0.05 * step)

        D1 = -1 * np.ones((1, self.Ns))
        D2 = np.zeros((1, self.Ns))
        D3 = np.ones((1, self.Ns))
        # TODO: should use online numbers?
        self.D = Pave * self.N * np.hstack((D1, D2, D3))
    
    def g_A(self):
        """
        Build A matrix.
        """
        # TODO: put the code here, consider the time interval of A


# initialize
sse = ev_ssm(N=50000, tnow=10)

# sse.act(t=300, tnow=10+1/12)

# sse.ev[sse.ev.u != sse.ev.u0]

# sse.act(t=300, tnow=10+1/12)
# sse.ev


EVA:
tnow=10 [24H]
50000 EVs, 9678 online, Total En=1249.73 MWh, SoC intervals: 20
Online En=242.05 MWh, mean SoC=30.0%


Calculate Mat. A, with interval 300s. from 10 to 11.

In [3]:
sse.reset(tnow=10)

for t in np.arange(10, 11.01, 300/3600):
    sse.act(t=300, tnow=t)

evr = sse.ev.copy()
evr['exl'] = evr['xl'].apply(lambda x: x[0] if len(x[0]) > 1 else None)
tm_train = evr[evr.exl.notna()].exl.to_list()

A = np.zeros((60, 60))
for data in tm_train:
    for idx in range(len(data)-1):
        A[data[idx], data[idx+1]] += 1
A = pd.DataFrame(A) # TODO: divide by row_sum
A = A.apply(lambda x: x/x.sum(), axis=1).fillna(0)

# A[A.columns[40:60]]

EVA: Successfully reset to time=10


Check A matrix

In [4]:
sse.tnow

11.083333333333341

In [5]:
x0 = list(sse.x.values())
sse.act(t=300, tnow=sse.tnow + 1/12)
x = list(sse.x.values())
xres = pd.DataFrame()
xres['x_comp'] = np.matmul(A, x0)
xres['x'] = x

In [6]:
xres

,x_comp,x
0,0.000000,0.00000
1,0.000000,0.00000
2,0.000013,0.00000
3,0.000217,0.00008
4,0.001337,0.00042
5,0.004692,0.00234
6,0.011461,0.00750
7,0.019996,0.01734
8,0.024963,0.02604
9,0.021888,0.02764


In [ ]:
# # --- plot online EV numbers ---
# # time = np.arange(0, 24, 0.1)
# # num = []
# # for t in time:
# #     sse.g_u(t)
# #     num.append(sse.ne)

# # fig, ax = plt.subplots()
# # ax.plot(time, num)
# # ax.set_title('Number of online EVs')
# # ax.set_xlabel('Time [h]')
# # ax.legend(['Online EV', 'Online capacity'])
# # ax.set_xlim(0, 24)

# # update g_x()

# # # reset sse
# # sse.reset(tnow=10)
# # sse.ev

# # --- Analytical method ---
# data = sse.ev.copy()
# data['dsc'] = data.Pc * data.nc / data.En / 6
# data['dsd'] = data.Pc * data.nc / data.En / 1
# data[['dsc', 'dsd']].plot(kind='kde')

# kde = stats.gaussian_kde(data.dsc)
# for i in range(-1,18,1):
#     lb = 0.05+0.05*i
#     ub = 0.1+0.05*i
#     ires = kde.integrate_box(lb, ub)
#     # print(np.round(ires, 4))

In [ ]:
# xdf = pd.DataFrame(xres)
# from hmmlearn import hmm
# gen_model = hmm.GaussianHMM(n_components=60, covariance_type="full")


# sse.reset(tnow=0)
# xres = []
# xres.append(sse.x.values())
# for t in np.arange(0, 24, 300/3600):
#     sse.act(t=300, tnow=t)
#     xres.append(sse.x.values())
# xdf = pd.DataFrame(xres)


# gen_model.fit(pd.concat([xdf]*100).reset_index(drop=True))
# gen_model.predict_proba(xdf.iloc[2].values.reshape(1, -1))

## SSM Matrix

In [ ]:
# get the x res by iterate EV from t=0 to t=14.01
# sse.reset(tnow=0)
# x_res = []
# for i, t in enumerate(np.arange(0, 14.01, 1/12)):
#     sse.act(t=300, tnow=t)
#     x_res.append(sse.x.tolist())

# pd.DataFrame(x_res).plot(legend=True)

## Simulation

Issues: how to integerate the SSM when using ADNES?

flow_chart:
```{python}
prep grid data:
ADNES: topology,  gen. limits, ramp. limits, line limits,
Outside: gen. cost, ramp. cost,

for $t_{OPF}$ in T (interval: 5min; total: 1h; [n=12]):
    aggregate EV data (from SSM), generate $PR_{e,i,u,t}$
    Do OPF, generate $PG_{i, t}$, $PR_{g, i, u, t}$, $PR_{g, i, d, t}$

    for t in $t_{OPF}$ (interval: 4s; total: 5min; [n=75]):
        Update data into dynamic system:
            # Note, constant power model should be used in TDS.
            # Use TimeSeries as the load. 
            power change: TGOV1.paux0
            load change: 

        Run TDS: generate SFR mileage
```

Co-Sim list:
```{python}
for $t_{OPF}$ in T:
    EVA report $pru_{max}$ $prd_{max}$, eqn xxx
    TCC do OPF, eqn xxx
    Assign dispatch signal to generation units

    for t in $t_{AGC}$:
        Assign AGC signal to AGC units
        Run TDS
```